## Incorporate time preference shocks

The code can be easily modified to incorporate time preference shocks. Please see the latest commit.

However, because the time preference shocks are estimated to be much more volatile than the consumption growth, we will have trouble getting an acurate estimate of $r(K)$ by simulation.

In [83]:
include("src/stability_coefficients.jl")

ez = EpsteinZinSSY()
cp = SSYConsumption()
θ, β, γ = ez.θ, ez.β, ez.γ;

In [78]:
@printf "%.4f" compute_spec_rad_coef(ez, cp, M=1000, N=1000)

0.9519

In [79]:
@printf "%.4f" compute_spec_rad_coef(ez, cp, M=1000, N=100000)

0.9949

In [80]:
@printf "%.4f" compute_spec_rad_coef(ez, cp, M=100000, N=1000)

0.9292

I even tried setting $M = N = 300000$. It ran for 2 hours and the results is $0.996$. It's hard to tell which one is more accurate. Fortunately, for an AR(1) time preference shock process, it seems that we can get an analytical solution.

## Analytical solutions

Since consumption growth and time preference shock are independent, the spectral radius can be written as:
\begin{align}
    r(K) &= \beta^\theta \lim_{N\to\infty} \left\{ \sup_{x\in\mathbb{X}} \mathbb{E}_x \exp\left[(1-\gamma) \ln \left(\frac{C_N}{C_0}\right) + \theta \ln\left(\frac{\lambda_N}{\lambda_0}\right) \right] \right\}^{1/N}\\
        &= \beta^\theta \lim_{N\to\infty} \left\{ \sup_{x\in\mathbb{X}} \mathbb{E}_x \exp\left[(1-\gamma) \ln \left(\frac{C_N}{C_0}\right)\right]  \sup_{x\in\mathbb{X}}\mathbb{E}_x \exp\left[\theta \ln\left(\frac{\lambda_N}{\lambda_0}\right) \right] \right\}^{1/N}\\
%        &= \beta^\theta \lim_{N\to\infty} \left\{ \lim_{M\to\infty} \frac{1}{M} \sum_{m=1}^M \exp\left[(1-\gamma) \sum_{n=1}^N \ln\left(\frac{C_n^m}{C_{n-1}^m}\right) + \theta \sum_{n=1}^N \ln \left(\frac{\lambda^m_n}{\lambda^m_{n-1}}\right) \right] \right\}^{1/N}\\
        &= r(\tilde{K}) \lim_{N\to\infty}\left\{\sup_{x\in\mathbb{X}}\mathbb{E}_x \exp\left[\theta \ln\left(\frac{\lambda_N}{\lambda_0}\right) \right] \right\}^{1/N}\\
\end{align}
where $\tilde{K}$ is the operator without time preference shocks. Assume that we know $r(\tilde{K})$.

Since $x_{n+1} := \ln (\lambda_{n+1}/\lambda_n) = \rho x_{n} + \sigma \eta_{n+1}$, we can write $\sum_{n=1}^N x_{n}$ in terms of $x_0$ and $(\eta_n)$:
\begin{align}
    x_{N} &= & \sigma \eta_N + \rho\sigma\eta_{N-1} + \rho^2\sigma\eta_{N-2} + \ldots + \rho^{N-1}\sigma\eta_1 &+ \rho^N x_{0}\\
    x_{N-1} &= & \sigma\eta_{N-1} + \rho\sigma\eta_{N-2} + \ldots + \rho^{N-2}\sigma\eta_1 &+ \rho^{N-1} x_{0}\\
    x_{N-2} &= & \sigma\eta_{N-2} + \ldots + \rho^{N-3}\sigma\eta_1 &+ \rho^{N-2} x_0\\
     \ldots & &\ldots&\\
    x_{1} &= & \sigma\eta_1 &+ \rho x_{0}
\end{align}
so 
\begin{equation}
    \sum_{n=1}^N x_{n} = \sigma\eta_N + (1+\rho)\sigma\eta_{N-1} + (1+\rho+\rho^2)\sigma\eta_{N-2} + \ldots + (1+\rho+\ldots+\rho^{N-1})\sigma\eta_1 + (\rho+\rho^2+\ldots+\rho^N)x_{0}.
\end{equation}

Hence, $\theta \ln(\lambda_N/\lambda_0)$ follows a normal distribution with
\begin{align}
    \mu_N &:= (\rho+\rho^2+\ldots+\rho^N)x_{0}\\
    \sigma_N^2 &:= \theta^2\sigma^2\left[1 + (1+\rho)^2 + \ldots + (1 + \rho + \ldots + \rho^{N-1})^2\right].
\end{align}
Then, as a lognormal distribution
$$\mathbb{E}_x \exp\left[\theta \ln\left(\frac{\lambda_N}{\lambda_0}\right) \right] = \exp\left(\mu_N + \sigma_N^2/2\right).$$
As long as $x_{0}$ is finite, as $N\to\infty$, the effect of $x_{0}$ dies out and we have
$$r(K)^{1/\theta} = r(\tilde{K})^{1/\theta} \left(\lim_{N\to\infty}e^{\sigma_N/2N}\right)^{1/\theta}.$$

I think the analytical solution of $r(\tilde{K})$ can be derived in a similar way, but I haven't tried. The estimated one seems accurate enough.

In [90]:
ρ, σ = cp.ρ_λ, cp.σ_λ
N = 10000000
variance = 1.0
for n in 2:N
    variance += (1-ρ^n)^2/(1-ρ)^2
end
variance = θ^2 * σ^2 * variance

@printf "%.4f" (compute_spec_rad_coef(ez, cp, time=false) * exp(variance/2N))^(1/θ)

0.1486

This is much smaller than the simulation results above. I thought I might have made an error somewhere, but I couldn't find any. My guess is that estimating the expectation of a lognormal random variable by simulation is very hard to achieve, especially for large $\sigma$. See the following example.

In [86]:
N = 100000000
σ = 10
srand(1234)
sim = σ*randn(N);
@printf "estimate: %19.4e\n" mean(exp.(sim))
@printf "theoretical mean: %11.4e\n" exp(σ^2/2)

estimate:          1.9130e+17
theoretical mean:  5.1847e+21


However, for smaller $\sigma$, simulation works just fine.

In [89]:
N = 1000000
σ = 1.5
srand(1234)
sim = σ*randn(N);
@printf "estimate: %15.4f\n" mean(exp.(sim))
@printf "theoretical mean: %7.4f\n" exp(σ^2/2)

estimate:          3.0874
theoretical mean:  3.0802
